In [ ]:
# conda install -c conda-forge geopandas

In [ ]:
# pip install geopy

In [297]:
import pandas as pd
import re
import copy

# https://www.geeksforgeeks.org/how-to-get-geolocation-in-python/
# https://www.geeksforgeeks.org/find-the-location-with-specified-latitude-and-longitude-using-python/
from geopy.geocoders import Nominatim
# calling the nominatim tool
geo_loc = Nominatim(user_agent="get_loc")

from geopy.extra.rate_limiter import RateLimiter

from geopy.distance import geodesic

In [2]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 300

In [60]:
df = pd.read_csv("Cyclistic_202010_202109.csv")
df.head()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13


In [4]:
df.isnull().sum()

Unnamed: 0                 0
ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    523467
start_station_id      523781
end_station_name      567268
end_station_id        567501
start_lat                  0
start_lng                  0
end_lat                 4821
end_lng                 4821
member_casual              0
riding_days                0
riding_time                0
dtype: int64

In [4]:
start_station = df.start_station_name.value_counts()
start_station_list = start_station.index.to_list()
len(start_station_list)

784

In [61]:
df['start_lat_short'] = df.start_lat.astype(str).str[:6]
df['start_lng_short'] = df.start_lng.astype(str).str[:7]
df['lat_lng_short'] = df['start_lat_short']+', '+df['start_lng_short']
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"


locations are a bit different, but similar

In [62]:
df = df.rename(columns = {'Unnamed: 0' : 'no'})
df.head()

,no,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"


In [13]:
df['lat_lng_short'].nunique()

2405

In [14]:
len(df)

5136261

In [15]:
df.duplicated('lat_lng_short').sum()

5133856

In [16]:
5136261 - 5133856

2405

In [63]:
df1 = df[:]

df1 = df1.drop_duplicates('lat_lng_short')
print(len(df1))
df1.tail()

2405


,no,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
5094558,5097862,7C5C4BFBB56779F6,electric_bike,2021-09-10 20:58:41,2021-09-10 21:13:18,Paulina St & 18th St,TA1307000159,NaN,NaN,41.858049,-87.669063,41.84,-87.72,member,0,00:14:37,41.858,-87.669,"41.858, -87.669"
5098438,5101742,A3DCC3B4E8387957,electric_bike,2021-09-08 08:50:54,2021-09-08 09:42:08,NaN,NaN,NaN,NaN,41.650000,-87.560000,41.57,-87.54,casual,0,00:51:14,41.65,-87.56,"41.65, -87.56"
5109326,5112630,8E34E64D48083921,electric_bike,2021-09-17 11:52:47,2021-09-17 12:30:34,NaN,NaN,NaN,NaN,41.830000,-87.710000,41.92,-87.70,member,0,00:37:47,41.83,-87.71,"41.83, -87.71"
5110369,5113673,5742BE0463806D7F,electric_bike,2021-09-29 15:14:31,2021-09-29 15:35:13,Noble St & Milwaukee Ave,13290,NaN,NaN,41.899018,-87.662521,41.87,-87.71,casual,0,00:20:42,41.899,-87.662,"41.899, -87.662"
5111917,5115221,A16CEA9F9596ED30,electric_bike,2021-09-06 11:25:04,2021-09-06 11:28:30,NaN,NaN,NaN,NaN,41.680000,-87.740000,41.69,-87.74,casual,0,00:03:26,41.68,-87.74,"41.68, -87.74"


In [64]:
df1 = df1.reset_index()
df1 = df1.drop(columns = ['no'])
df1

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,5094558,7C5C4BFBB56779F6,electric_bike,2021-09-10 20:58:41,2021-09-10 21:13:18,Paulina St & 18th St,TA1307000159,NaN,NaN,41.858049,-87.669063,41.840000,-87.720000,member,0,00:14:37,41.858,-87.669,"41.858, -87.669"
2401,5098438,A3DCC3B4E8387957,electric_bike,2021-09-08 08:50:54,2021-09-08 09:42:08,NaN,NaN,NaN,NaN,41.650000,-87.560000,41.570000,-87.540000,casual,0,00:51:14,41.65,-87.56,"41.65, -87.56"
2402,5109326,8E34E64D48083921,electric_bike,2021-09-17 11:52:47,2021-09-17 12:30:34,NaN,NaN,NaN,NaN,41.830000,-87.710000,41.920000,-87.700000,member,0,00:37:47,41.83,-87.71,"41.83, -87.71"
2403,5110369,5742BE0463806D7F,electric_bike,2021-09-29 15:14:31,2021-09-29 15:35:13,Noble St & Milwaukee Ave,13290,NaN,NaN,41.899018,-87.662521,41.870000,-87.710000,casual,0,00:20:42,41.899,-87.662,"41.899, -87.662"


## skip

Separate df_lat_lng into three as geo_loc.geocode() has a access time limit

In [65]:
df_lat_lng1 = df1.loc[:802]
df_lat_lng2 = df1.loc[803:1605]
df_lat_lng3 = df1.loc[1606:]

print(len(df_lat_lng1),
      len(df_lat_lng2),
      len(df_lat_lng3)
     )

803 803 799


In [240]:
# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")

loc_list1 = []
loc_list2 = []
loc_list3 = []

# As df_lat_lng1 start from 0, I don't need to set the range specifically
for i in range(len(df_lat_lng1)):
    zip_code = geo_loc.geocode(df_lat_lng1['lat_lng_short'][i]).address
    loc_list1.append(zip_code)
print('df_lat_lng1 - Done')

# Set the range as df_lat_lng2 starts from 802 and ends at 1604
for j in range(df_lat_lng2['lat_lng_short'].index.start, df_lat_lng2['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng2['lat_lng_short'][j]).address
    loc_list2.append(zip_code)
print('df_lat_lng2 - Done')

# Set the range as df_lat_lng3 starts from 1605 and ends at 2404
for k in range(df_lat_lng3['lat_lng_short'].index.start, df_lat_lng3['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng3['lat_lng_short'][k]).address
    loc_list3.append(zip_code)
print('df_lat_lng3 - Done')

for i in range(3)
    for j in range()

df_lat_lng1 - Done
df_lat_lng2 - Done
df_lat_lng3 - Done


# End lat lng

Take the integer part

1. [start:end]
2. [start:end]
   
   ...
   
Last. [start:]

In [436]:
df3['end_lat_lng'].nunique()/800 # 6801

8.50125

In [141]:
# Set segement size
seg_size = 800
# Take only integer part
integer_part = df3['end_lat_lng'].nunique()//seg_size
# Set 0 for start and seg_size (800) for end
start = 0
end = seg_size

seg_list = []
index_list = []

# The range is integer_part + 1, which is 8 + 1. The last loop 1 is for residual numbers. 
for i in range(integer_part+1):
    if i != integer_part:
        seg_list.append(df3.loc[start:end])
        index_list.append(seg_list[i].index)
        start = end + 1
        end = end + seg_size    
    else:
        seg_list.append(df3.loc[start:])
        index_list.append(seg_list[i].index)

In [ ]:
# Get addresses 
add_list = []
# 1704 - 1750
for i in range(len(index_list)):
    for j in range(index_list[i].start, index_list[i].stop):
#         zip_code_list.append(geo_loc.reverse(df3.loc[j, 'end_lat_lng']).raw['address']['postcode'])
        add_list.append(geo_loc.geocode(seg_list[i].loc[j, 'end_lat_lng']).address)    

In [ ]:
comb_add_list = []

for i in range(len(add_list)):
    comb_add_list += add_list[i]
len(comb_add_list)

In [449]:
df3['end_address'] = comb_add_list
df3.head()

,index,Start Lat,Start Lng,Zip Code,End Lat,End Lng,start_lat_lng,end_lat_lng,end_address
0,0,42.064313,-87.686152,60201,42.064313,-87.686152,"42.064, -87.686","42.064, -87.686","Central & Girard (Purple Line) EB, Central Str..."
1,1,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677","Parkes Hall, 1870, Sheridan Road, Downtown, Ev..."
2,4,42.064313,-87.686152,60201,42.032562,-87.679101,"42.064, -87.686","42.032, -87.679","Firehouse Grill, 750, Chicago Avenue, South Ev..."
3,6,42.063598,-87.69873,60201,42.063598,-87.69873,"42.063, -87.698","42.063, -87.698","Verizon, 2516, Green Bay Road, Evanston, Evans..."
4,10,42.063598,-87.69873,60201,41.96167,-87.65464,"42.063, -87.698","41.961, -87.654","4315-4317, North Broadway, Buena Park, Uptown,..."


In [450]:
df3.isnull().sum()

index            0
Start Lat        0
Start Lng        0
Zip Code         0
End Lat          0
End Lng          0
start_lat_lng    0
end_lat_lng      0
end_address      0
dtype: int64

In [451]:
df3.to_csv('df3_end_add_n_distance.csv')

In [452]:
df3['end_address']

0       Central & Girard (Purple Line) EB, Central Str...
1       Parkes Hall, 1870, Sheridan Road, Downtown, Ev...
2       Firehouse Grill, 750, Chicago Avenue, South Ev...
3       Verizon, 2516, Green Bay Road, Evanston, Evans...
4       4315-4317, North Broadway, Buena Park, Uptown,...
                              ...                        
6796    6817-6819, North Wayne Avenue, Rogers Park, Ch...
6797    1020, West Belmont Avenue, Northalsted, Lake V...
6798    3656, North Fremont Street, Northalsted, Lake ...
6799    1423-1427, West Estes Avenue, Rogers Park, Chi...
6800    South Shore Golf Course, Lakefront Trail, Sout...
Name: end_address, Length: 6801, dtype: object

In [471]:
# for i in range(len(df3)):
df3['zip_code'] = df3['end_address'].apply(lambda x: x.split(',')[-2].strip())
df3.head()

,index,Start Lat,Start Lng,Zip Code,End Lat,End Lng,start_lat_lng,end_lat_lng,end_address,zip_code
0,0,42.064313,-87.686152,60201,42.064313,-87.686152,"42.064, -87.686","42.064, -87.686","Central & Girard (Purple Line) EB, Central Str...",60201
1,1,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677","Parkes Hall, 1870, Sheridan Road, Downtown, Ev...",60201
2,4,42.064313,-87.686152,60201,42.032562,-87.679101,"42.064, -87.686","42.032, -87.679","Firehouse Grill, 750, Chicago Avenue, South Ev...",60202
3,6,42.063598,-87.69873,60201,42.063598,-87.69873,"42.063, -87.698","42.063, -87.698","Verizon, 2516, Green Bay Road, Evanston, Evans...",60201
4,10,42.063598,-87.69873,60201,41.96167,-87.65464,"42.063, -87.698","41.961, -87.654","4315-4317, North Broadway, Buena Park, Uptown,...",60613


In [478]:
df3['zip_code'].unique()

array(['60201', '60202', '60613', '60626', '60208', '60657', '60660',
       '60640', '60645', '9020', '60611', '60618', '60605', '60625',
       '60632', '60659', '60601', '60614', '60651', '60602', '60647',
       '60615', '60642', '60610', '60654', '60606', '60643', '60622',
       '60617', '46394', '60661', '60616', '60653', 'IL', '60623',
       '60603', '60607', '60302', '60608', '60604', '60641', '60609',
       '60622-4580', '60619', '60409', '60646', '60637', '60612', '60649',
       '60636', '60629', '60617:60649', '60624', '60712', 'Illinois',
       '60620', '60655', '60203', '60644', '60630', '60638', '90615',
       '60621', '60628', '60633', '60827-6427', '60827', '60639', '60696',
       '60652', '60804', '60091', '60634', '60805', '60803', '60453',
       '46327', '60193', '60076', '60419', '60411', '60402', '60631',
       '60406', '60304', '46320', '60706', '60093', '60656', '60022',
       '46312', '60707', '60053', '60301', '60082', '60153', '60104',
       '60077'

In [483]:
len(df3['zip_code'][0])

5

In [484]:
wrong_dict = {'index' : [], 'start_lat_lng': [], 'end_lat_lng': [], 'address' : [], 'zip_code' : []}

for i in range(len(df3)):
    if (len(df3['zip_code'][i]) != 5) & (re.search('^60', df3['zip_code'][i]) == None):


14
94
320
340
350
402
431
442
464
523
594
616
657
712
865
931
964
972
1014
1204
1340
1465
1495
1633
1668
1669
1681
1686
1697
1720
1733
1788
1790
1909
1922
1956
1970
2184
2224
2244
2270
2301
2462
2496
2521
2524
2587
2645
2657
2670
2798
2858
2877
2922
2978
2994
3004
3036
3075
3141
3180
3181
3216
3226
3284
3312
3317
3415
3432
3469
3485
3513
3519
3562
3579
3611
3642
3648
3682
3731
3747
3749
3762
3784
3823
3892
3994
4050
4074
4155
4167
4183
4257
4324
4376
4461
4563
4566
4568
4581
4612
4684
4867
4874
4893
4941
5007
5059
5075
5088
5121
5158
5238
5281
5284
5373
5389
5486
5504
5546
5632
5669
5734
5756
5783
5851
5952
5962
6084
6107
6133
6192
6406
6420
6427
6436
6460
6472
6525
6540
6645


In [ ]:
# lists in a dictionary
wrong_dict = {'index' : [], 'start_lat_lng': [], 'end_lat_lng': [], 'address' : [], 'zip_code' : []}

for i in range(len(df1)):
    # Find any zip code not starting with 60
    if re.search('^60', df1['zip_code'][i]) == None:
#     if re.search('.60', df1['full_address'][i]) == None: # wrong
        # Store their index, start_lat_lng, end_lat_lng, and full_address into err_dict
        wrong_dict['index'].append(df1['index'][i])
        wrong_dict['start_lat_lng'].append(df1['start_lat_lng'][i])
        wrong_dict['end_lat_lng'].append(df1['end_lat_lng'][i])
        wrong_dict['address'].append(df1['full_address'][i])
        wrong_dict['zip_code'].append(df1['zip_code'][i])
wrong_dict

# df2

In [437]:
df2['start_lat_lng'] = df2['Start Lat'].astype(str).str[:10]+', '+df2['Start Lng'].astype(str).str[:11]
df2['end_lat_lng'] = df2['End Lat'].astype(str).str[:10]+', '+df2['End Lng'].astype(str).str[:11]
df2.head(1)

,day_of_week,End Station Id,End Station Name,Ended At,Member Casual,Ride Id,Rideable Type,Riding Time,season,Start Station Id,Start Station Name,Started At,Zip Code,classic_bike,docked_bike,electrict_bike,End Lat,End Lng,F1,Riding Days,riding_time,Start Lat,Start Lng,test,start_lat_lng,end_lat_lng,distance_m
0,Friday,NaN,Central St & Girard Ave,26/03/2021 4:28:58 PM,casual,56A8B5FB254C94A8,classic_bike,30/12/1899 12:02:15 AM,SPRING,NaN,Central St & Girard Ave,26/03/2021 4:26:43 PM,60201,100,0,0,42.064313,-87.686152,1069469,0,2.25,42.064313,-87.686152,12,"42.064313, -87.686152","42.064313, -87.686152",0.0


In [427]:
# Calculate the distance between coordinates
dist_list = []
# 2008 - 2037
for i in range(len(df2)):
    dist_list.append(geodesic(df2.loc[i, 'start_lat_lng'], df2.loc[i, 'end_lat_lng']).m)

In [428]:
len(dist_list)

5129633

In [435]:
df2['distance_m'] = dist_list
df2['distance_m'] = round(df2['distance_m'],2)
df2.tail()

,day_of_week,End Station Id,End Station Name,Ended At,Member Casual,Ride Id,Rideable Type,Riding Time,season,Start Station Id,Start Station Name,Started At,Zip Code,classic_bike,docked_bike,electrict_bike,End Lat,End Lng,F1,Riding Days,riding_time,Start Lat,Start Lng,test,start_lat_lng,end_lat_lng,distance_m
5129628,Thursday,13084.0,California Ave & Milwaukee Ave,2/09/2021 7:07:48 PM,member,058A0C687460B27D,electric_bike,30/12/1899 12:32:06 AM,SUMMER,NaN,NaN,2/09/2021 6:35:42 PM,60707,0,0,100,41.922611,-87.697170,4859110,0,32.100000,41.93,-87.79,13,"41.93, -87.79","41.922611, -87.6971705",7743.48
5129629,Tuesday,13084.0,California Ave & Milwaukee Ave,28/09/2021 7:57:39 PM,member,F031FE67D211249E,electric_bike,30/12/1899 12:33:03 AM,SUMMER,NaN,NaN,28/09/2021 7:24:36 PM,60707,0,0,100,41.922691,-87.697177,4858710,0,33.050000,41.93,-87.79,13,"41.93, -87.79","41.9226913, -87.697177",7742.00
5129630,Wednesday,13084.0,California Ave & Milwaukee Ave,1/09/2021 7:39:23 PM,member,876F51E3CDABAFE8,electric_bike,30/12/1899 12:34:44 AM,SUMMER,NaN,NaN,1/09/2021 7:04:39 PM,60707,0,0,100,41.922633,-87.697153,4859109,0,34.733333,41.92,-87.79,13,"41.92, -87.79","41.9226328, -87.6971535",7707.42
5129631,Wednesday,20206.0,Homewood Ave & 115th St,29/09/2021 4:54:36 PM,member,305925F2012483D3,electric_bike,30/12/1899 12:38:53 AM,SUMMER,NaN,NaN,29/09/2021 4:15:43 PM,60805,0,0,100,41.684572,-87.670796,4859510,0,38.883333,41.71,-87.70,13,"41.71, -87.7","41.6845721, -87.6707958",3726.40
5129632,Tuesday,20228.0,Constance Ave & 95th St,21/09/2021 4:41:38 PM,member,492F760A0095D654,electric_bike,30/12/1899 12:38:28 AM,SUMMER,NaN,NaN,21/09/2021 4:03:10 PM,60827,0,0,100,41.722100,-87.578979,4936161,0,38.466667,41.65,-87.62,13,"41.65, -87.62","41.7221001, -87.5789793",8705.82


df2.loc[(df2['start_lat_lng'].str.len() < 21) | (df2['end_lat_lng'].str.len() < 21)]

In [486]:
df2.to_csv('df2_distance.csv')

# df2 END

In [322]:
# test = pd.DataFrame({'name': ['42.064, -87.686', 'berlin', 'london']})
test = pd.DataFrame({'name': ['42.064, -87.686', '42.064, -87.686', '42.064, -87.686']})

from geopy.extra.rate_limiter import RateLimiter
# geo_loc.reverse('42.064, -87.686').raw['address']['postcode']
# geocode = RateLimiter(get_loc.geocode, min_delay_seconds=1)
geo_reverse = RateLimiter(get_loc.reverse, min_delay_seconds=1)
test['location'] = test['name'].apply(geocode)

# df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [276]:
geo_loc.geocode(seg_list[0].loc[0, 'end_lat_lng']).raw

{'place_id': 41026435,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 3266573885,
 'boundingbox': ['42.0640714', '42.0641714', '-87.6859784', '-87.6858784'],
 'lat': '42.0641214',
 'lon': '-87.6859284',
 'display_name': 'Central & Girard (Purple Line) EB, Central Street, Evanston, Evanston Township, Cook County, Illinois, 60201, United States',
 'class': 'highway',
 'type': 'bus_stop',
 'importance': 0.001,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//transport_bus_stop2.p.20.png'}

In [249]:
geo_loc.geocode(seg_list[0].loc[0, 'end_lat_lng']).point

Point(42.0641214, -87.6859284, 0.0)

Merge loc_list1, 2 and 3 into loc_list and add it to df_lat_lng['full_address']. 

I saved this df because to avoid the above time-consuming process.

In [261]:
loc_list = loc_list1 + loc_list2 + loc_list3
df1['full_address'] = loc_list
df1.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,full_address
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638","325, West Fullerton Parkway, Mid-North Distric..."
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663","3630, North Wayne Avenue, Wrigleyville, Lake V..."
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585","6717, South Cornell Avenue, Parkside, South Sh..."
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659","3740-3742, North Clark Street, Wrigleyville, L..."
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663","2533, North Southport Avenue, Lincoln Park, Ch..."


## to here

In [73]:
# df1.to_csv('df_lat_lng.csv')
df1 = pd.read_csv('df_lat_lng.csv')
df1 = df1.drop(columns = {'Unnamed: 0'})
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               2405 non-null   int64  
 1   ride_id             2405 non-null   object 
 2   rideable_type       2405 non-null   object 
 3   started_at          2405 non-null   object 
 4   ended_at            2405 non-null   object 
 5   start_station_name  1742 non-null   object 
 6   start_station_id    1742 non-null   object 
 7   end_station_name    1564 non-null   object 
 8   end_station_id      1562 non-null   object 
 9   start_lat           2405 non-null   float64
 10  start_lng           2405 non-null   float64
 11  end_lat             2405 non-null   float64
 12  end_lng             2405 non-null   float64
 13  member_casual       2405 non-null   object 
 14  riding_days         2405 non-null   int64  
 15  riding_time         2405 non-null   object 
 16  start_

In [74]:
df1['start_lat_lng'] = df1['start_lat'].astype(str).str[:10]+","+df1['start_lng'].astype(str).str[:11]
df1['end_lat_lng'] = df1['end_lat'].astype(str).str[:10]+","+df1['end_lng'].astype(str).str[:11]
df1['zip_code'] = df1['full_address'].str[-20:-15]
df1.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,full_address,start_lat_lng,end_lat_lng,zip_code
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638","325, West Fullerton Parkway, Mid-North Distric...","41.9261008,-87.6389773","41.8903451,-87.626068",60614
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663","3630, North Wayne Avenue, Wrigleyville, Lake V...","41.9481723,-87.663911","41.9295276,-87.7078178",60613
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585","6717, South Cornell Avenue, Parkside, South Sh...","41.7734621,-87.5853716","41.7914546,-87.6000496",60649
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659","3740-3742, North Clark Street, Wrigleyville, L...","41.9508548,-87.6592443","41.952809,-87.6501031",60613
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663","2533, North Southport Avenue, Lincoln Park, Ch...","41.9288568,-87.663962","41.9177768,-87.6914336",60613


In [75]:
df1['zip_code'].value_counts()

60611    143
60614    100
60610     90
60601     82
60605     71
60617     69
60606     68
60607     68
60654     63
60622     62
60625     61
60613     61
60661     57
60612     56
60628     53
60657     52
60608     51
60637     50
60616     49
60620     49
60640     44
60619     43
60647     42
60609     42
60643     35
60604     34
60618     33
60615     33
60602     33
60626     31
60645     30
60624     28
60623     27
60632     26
60603     25
60629     25
60649     25
60634     24
60636     24
60651     24
60653     23
60644     23
60621     23
60641     22
60660     20
60642     19
60202     18
60201     18
60633     16
60655     16
s, IL     16
60630     13
60639     12
60659     12
60302     11
60827     10
 9020     10
60804     10
inois     10
60707      9
60208      9
46394      9
60696      8
60646      8
60652      7
60712      6
60638      6
60406      6
60803      6
60076      6
60091      6
90615      5
60402      5
60203      4
-4580      4
60409      4
60805      4

In [76]:
df1['zip_code'].nunique()

83

Some texts don't seem to have the zip code
 
* s, IL     16
* inois     10
*  9020     10
* 46394      9
* 90615      5
* -4580      4
* -6427      2
* 46320      1

Some of them are still in the US, but not in Illinois.

Some of them don't have a zip code at all.

## Wrong zip codes

The zip codes in Chicago starts with 60-, so I used regular expressions to find any zip codes that don't start with 60.


In [50]:
# https://www.guru99.com/python-dictionary-append.html
# lists in a dictionary
wrong_dict = {'index' : [], 'start_lat_lng': [], 'end_lat_lng': [], 'address' : [], 'zip_code' : []}

# https://www.geeksforgeeks.org/python-regex-cheat-sheet/
# https://pythonexamples.org/python-regex-check-if-string-starts-with-specific-word/
for i in range(len(df1)):
    # Find any zip code not starting with 60
    if re.search('^60', df1['zip_code'][i]) == None:
#     if re.search('.60', df1['full_address'][i]) == None: # wrong
        # Store their index, start_lat_lng, end_lat_lng, and full_address into err_dict
        wrong_dict['index'].append(df1['index'][i])
        wrong_dict['start_lat_lng'].append(df1['start_lat_lng'][i])
        wrong_dict['end_lat_lng'].append(df1['end_lat_lng'][i])
        wrong_dict['address'].append(df1['full_address'][i])
        wrong_dict['zip_code'].append(df1['zip_code'][i])
wrong_dict

{'index': [13,
  45,
  86,
  144,
  366,
  372,
  489,
  538,
  870,
  1055,
  1195,
  1350,
  1417,
  1638,
  1785,
  2225,
  2277,
  3094,
  3384,
  3645,
  3815,
  5358,
  6430,
  6499,
  7019,
  8841,
  9266,
  10727,
  10740,
  14526,
  14624,
  16412,
  25073,
  30244,
  33365,
  35668,
  42943,
  55094,
  64571,
  65674,
  403518,
  540503,
  725676,
  943854,
  1181800,
  1187825,
  1457772,
  2271890,
  2853240,
  3248483,
  3569667,
  3750105,
  4379438,
  4387295,
  4527112,
  4902131,
  5090477],
 'start_lat_lng': ['41.9691073,-87.6742151',
  '41.67,-87.66',
  '41.96,-87.64',
  '41.69,-87.53',
  '41.9633206,-87.6793091',
  '41.9098433,-87.7053461',
  '41.83,-87.6',
  '41.86,-87.61',
  '41.9671258,-87.6790515',
  '41.956048,-87.6804296',
  '41.8513816,-87.6188563',
  '41.8838115,-87.6238975',
  '41.8525165,-87.6265323',
  '41.7935036,-87.6158841',
  '41.8603026,-87.6259061',
  '41.6849586,-87.6453421',
  '41.8920213,-87.6894271',
  '41.853177,-87.6318415',
  '41.69,-87.52',


**df_wr_dict** contains the data with the wrong zip codes

In [51]:
# Convert err_dict into a DataFrame. easily accessible and looking better
df_wr_dict = pd.DataFrame(wrong_dict)
print(len(df_wr_dict))
df_wr_dict.head()

57


,index,start_lat_lng,end_lat_lng,address,zip_code
0,13,"41.9691073,-87.6742151","41.8816883,-87.6294188","4801-4809, North Ravenswood Avenue, Andersonvi...",9020
1,45,"41.67,-87.66","41.71,-87.57","West Pullman, Calumet Park, Calumet Township, ...",-6427
2,86,"41.96,-87.64","41.9639225,-87.6381561","West Montrose Harbor Drive, Uptown, Chicago, L...",inois
3,144,"41.69,-87.53","41.69,-87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...",46394
4,366,"41.9633206,-87.6793091","41.9394305,-87.7116701","4449-4455, North Damen Avenue, West Ravenswood...",9020


In [77]:
df1['zip_code'].unique()

array(['60614', '60613', '60649', '60610', '60611', '60651', '60606',
       '60603', ' 9020', '60617', '60622', '60626', '60619', '60608',
       '60612', '60642', '60640', '-6427', '60601', '60657', '60641',
       '60647', '60632', '60602', 'inois', '60654', '60645', '60660',
       '60637', '60605', '60625', '60616', '60618', '60201', '60202',
       '46394', '60607', '60646', '60628', '60208', '60615', '60623',
       '60661', '60604', '60302', '60639', '-4580', '60624', '60696',
       '60653', '60636', '60620', '60659', '60629', '60621', '60609',
       '60644', '60630', 's, IL', '60643', '60638', '60409', '60633',
       '60827', '60712', '60652', '46320', '60634', '60203', '90615',
       '60655', '60076', '60402', '60091', '60707', '60804', '60803',
       '60805', '60406', '60453', '60472', '60176', '60304'], dtype=object)

In [53]:
df_wr_dict['zip_code'].value_counts()

s, IL    16
 9020    10
inois    10
46394     9
90615     5
-4580     4
-6427     2
46320     1
Name: zip_code, dtype: int64

In [78]:
df_wr_dict.isnull().sum()

index            0
start_lat_lng    0
end_lat_lng      0
address          0
zip_code         0
new_address      0
dtype: int64

If the coordinate doesn't have any zip code, then choose the nearest shop to get the zip code. 

In [54]:
wr_dict_new_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
                   '12303 S Ashland Ave, Calumet Park, IL 60827, USA',
 'Uptown, Chicago, IL 60613, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
'West Town, Chicago, IL 60651, USA',
 '3155 S Lake Shore Dr, Chicago, IL 60616, United States',
 'Parking lot, Chicago, IL 60605, USA',
 '4705 N Damen Ave, Chicago, IL 60625, USA',
 '4108 N Lincoln Ave, Chicago, IL 60618, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 'Millennium Park, Chicago, IL 60601, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'MLK Jr Dr & 56th St, Chicago, IL 60612, United States',
 'Parking lot, Chicago, IL 60605, USA',
'Major Taylor Trail & 115th St, Chicago, IL 60643, USA',
'Smith Park, Chicago, IL 60612, USA',
 '2130 S Wentworth Ave, Chicago, IL 60616, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '10733 S Ave F, Chicago, IL 60617, USA',
 'Armour Square, Chicago, IL 60616, USA',
 'E 47th St, Chicago, IL 60653, USA',
'Smith Park, Chicago, IL 60612, USA',
 '72 W Monroe St, Chicago, IL 60603, United States',
 '5410 N Damen Ave, Chicago, IL 60625, USA',
 '3050 Moe Dr, Chicago, IL 60616, USA',
 'Baseball Junior/Softball, Chicago, IL 60615, USA',
 'Bronzeville, Chicago, IL 60653, USA',
 'State St & 95th St, Chicago, IL 60619, United States',
 '5074 S Calumet Ave, Chicago, IL 60612, USA',
 'Chicago, IL 60616, United States',
 'East Side, Chicago, IL 60617, USA',
 '100 W Randolph St 2nd fl, Chicago, IL 60601, United States',
'2910 W Ferdinand St, Chicago, IL 60612, USA',
 '1736 S Michigan Ave, Chicago, IL 60616, USA',
 '4802 N Ravenswood Ave, Chicago, IL 60612, USA',
'Parking lot, Chicago, IL 60605, USA',
 'Chicago, IL 60640, United States',
 'Dan Ryan Woods Trails, Chicago, IL 60620, USA',
 '2150 W Lawrence Ave, Chicago, IL 60625, USA',
 'Damen Ave & Leland Ave, Chicago, IL 60625, United States',
 'Chicago, IL 60605, United States',
 'McCormick Place, Chicago, IL 60616, United States',
 'Wells St & 19th St, Chicago, IL 60616, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 'Lincoln Ave & Sunnyside Ave, Chicago, IL 60625, United States',
 '1719 S Clinton St, Chicago, IL 60616, USA',
 'Museum Campus, Chicago, IL 60605, USA',
 'Parking lot, Chicago, IL 60616, USA',
 'Chicago, IL 60633, United States',
 '338 E 51st St, Chicago, IL 60615, USA',
 'S Wolf Lake Blvd, Chicago, IL 60633, United States',
 'Chicago, IL 60633, United States',
 '5420-5400 N Hoyne Ave, Chicago, IL 60625, USA',
 '1400-1414 Michigan Ave, Chicago, IL 60605, USA',
 'Chicago, IL 60633, USA']

In [55]:
df_wr_dict['new_address'] = wr_dict_new_add
# df_wr_dict = df_wr_dict.drop(columns = 'new_add')
df_wr_dict.head()

,index,start_lat_lng,end_lat_lng,address,zip_code,new_address
0,13,"41.9691073,-87.6742151","41.8816883,-87.6294188","4801-4809, North Ravenswood Avenue, Andersonvi...",9020,"Ravenswood Ave & Lawrence Ave, Chicago, IL 606..."
1,45,"41.67,-87.66","41.71,-87.57","West Pullman, Calumet Park, Calumet Township, ...",-6427,"12303 S Ashland Ave, Calumet Park, IL 60827, USA"
2,86,"41.96,-87.64","41.9639225,-87.6381561","West Montrose Harbor Drive, Uptown, Chicago, L...",inois,"Uptown, Chicago, IL 60613, USA"
3,144,"41.69,-87.53","41.69,-87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...",46394,"11251-11249 S Ave B, Chicago, IL 60617, USA"
4,366,"41.9633206,-87.6793091","41.9394305,-87.7116701","4449-4455, North Damen Avenue, West Ravenswood...",9020,"2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA"


In [62]:
# Replace United States to USA
df_wr_dict['new_address'] = df_wr_dict['new_address'].replace('.United States', ' USA', regex=True)

# Check if all the word have changed 
for i in range(len(df_wr_dict)):
    if re.search('.United States', df_wr_dict['new_address'][i]) != None:
        print(i)


In [61]:
df_wr_dict['new_address'].head()

0    Ravenswood Ave & Lawrence Ave, Chicago, IL 606...
1     12303 S Ashland Ave, Calumet Park, IL 60827, USA
2                       Uptown, Chicago, IL 60613, USA
3          11251-11249 S Ave B, Chicago, IL 60617, USA
4    2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA
Name: new_address, dtype: object

In [63]:
# Check the new zip codes
df_wr_dict['zip_code'] = df_wr_dict['new_address'].str[-10:-5]
df_wr_dict['zip_code'].value_counts()

60616    12
60625     7
60612     6
60617     6
60605     6
60633     4
60615     2
60653     2
60640     2
60601     2
60827     1
60618     1
60643     1
60651     1
60603     1
60613     1
60619     1
60620     1
Name: zip_code, dtype: int64

In [64]:
df_wr_dict.head()

,index,start_lat_lng,end_lat_lng,address,zip_code,new_address
0,13,"41.9691073,-87.6742151","41.8816883,-87.6294188","4801-4809, North Ravenswood Avenue, Andersonvi...",60640,"Ravenswood Ave & Lawrence Ave, Chicago, IL 606..."
1,45,"41.67,-87.66","41.71,-87.57","West Pullman, Calumet Park, Calumet Township, ...",60827,"12303 S Ashland Ave, Calumet Park, IL 60827, USA"
2,86,"41.96,-87.64","41.9639225,-87.6381561","West Montrose Harbor Drive, Uptown, Chicago, L...",60613,"Uptown, Chicago, IL 60613, USA"
3,144,"41.69,-87.53","41.69,-87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...",60617,"11251-11249 S Ave B, Chicago, IL 60617, USA"
4,366,"41.9633206,-87.6793091","41.9394305,-87.7116701","4449-4455, North Damen Avenue, West Ravenswood...",60625,"2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA"


In [66]:
print(len(df1), # 2405
      len(df_wr_dict)) # 57

2405 57


In [67]:
df_wr_dict.isnull().sum()

index            0
start_lat_lng    0
end_lat_lng      0
address          0
zip_code         0
new_address      0
dtype: int64

In [79]:
print(df1['zip_code'].nunique())
print(df1['zip_code'].unique())

83
['60614' '60613' '60649' '60610' '60611' '60651' '60606' '60603' ' 9020'
 '60617' '60622' '60626' '60619' '60608' '60612' '60642' '60640' '-6427'
 '60601' '60657' '60641' '60647' '60632' '60602' 'inois' '60654' '60645'
 '60660' '60637' '60605' '60625' '60616' '60618' '60201' '60202' '46394'
 '60607' '60646' '60628' '60208' '60615' '60623' '60661' '60604' '60302'
 '60639' '-4580' '60624' '60696' '60653' '60636' '60620' '60659' '60629'
 '60621' '60609' '60644' '60630' 's, IL' '60643' '60638' '60409' '60633'
 '60827' '60712' '60652' '46320' '60634' '60203' '90615' '60655' '60076'
 '60402' '60091' '60707' '60804' '60803' '60805' '60406' '60453' '60472'
 '60176' '60304']


In [69]:
print(df_wr_dict['zip_code'].nunique())
print(df_wr_dict['zip_code'].unique())

18
['60640' '60827' '60613' '60617' '60625' '60651' '60616' '60605' '60618'
 '60601' '60612' '60643' '60653' '60603' '60615' '60619' '60620' '60633']


In [642]:
df_wr_dict['index'].unique()

array([     13,      45,      86,     144,     366,     372,     489,
           538,     870,    1055,    1195,    1350,    1417,    1638,
          1785,    2225,    2277,    3094,    3384,    3645,    3815,
          5358,    6430,    6499,    7019,    8841,    9266,   10727,
         10740,   14526,   14624,   16412,   25073,   30244,   33365,
         35668,   42943,   55094,   64571,   65674,  403518,  540503,
        725676,  943854, 1181800, 1187825, 1457772, 2271890, 2853240,
       3248483, 3569667, 3750105, 4379438, 4387295, 4527112, 4902131,
       5090477], dtype=int64)

## When dealing with a copy of df

https://towardsdatascience.com/how-to-suppress-settingwithcopywarning-in-pandas-c0c759bd0f10

The 1. and 2. code do the same thing, but the first code gives a warning message, while the second dosen't. 

1. df1['zip_code'].loc[df1['index'] == i] = df_wr_dict['zip_code'].loc[df_wr_dict['index'] == i][row_no]
2. df1.loc[df1['index'] == i, 'zip_code'] = df_wr_dict.loc[df_wr_dict['index'] == i, 'zip_code'][row_no]

In [142]:
# To extract a value from selected entities, I need to specify their row numbers
# Wihout out it, it returns nan 
row_no = 0

for i in df_wr_dict['index']:
#     df1['zip_code'].loc[df1['index'] == i] = df_wr_dict['zip_code'].loc[df_wr_dict['index'] == i][row_no]
    df1.loc[df1['index'] == i, 'zip_code'] = df_wr_dict.loc[df_wr_dict['index'] == i, 'zip_code'][row_no]
    row_no += 1

In [143]:
# At last, I have all zip codes that I need 
print(df1['zip_code'].nunique())
print(df1['zip_code'].unique())

75
['60614' '60613' '60649' '60610' '60611' '60651' '60606' '60603' '60640'
 '60617' '60622' '60626' '60619' '60608' '60612' '60642' '60827' '60601'
 '60657' '60641' '60647' '60632' '60602' '60654' '60645' '60660' '60637'
 '60605' '60625' '60616' '60618' '60201' '60202' '60607' '60646' '60628'
 '60208' '60615' '60623' '60661' '60604' '60302' '60639' '60624' '60696'
 '60653' '60636' '60620' '60659' '60629' '60621' '60609' '60644' '60630'
 '60643' '60638' '60409' '60633' '60712' '60652' '60634' '60203' '60655'
 '60076' '60402' '60091' '60707' '60804' '60803' '60805' '60406' '60453'
 '60472' '60176' '60304']


In [83]:
df1.to_csv('df1_code.csv')

# Fill the zip code to the whole DF

As I used the unique values in the column *lat_lng_short to find zip code, 

based on the values in *lat_lng_short* in *df1* match the values in *lat_lng_short* in *df2*

In [157]:
# Make df2 a copy of df
df2 = df.loc[:]
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5136261 entries, 0 to 5136260
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   no                  int64  
 1   ride_id             object 
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  riding_days         int64  
 15  riding_time         object 
 16  start_lat_short     object 
 17  start_lng_short     object 
 18  lat_lng_short       object 
dtypes: float64(4), int64(2), object(13)
memory usage: 744.5+ MB


In [173]:
for i in range(1):
    print(i)

0


In [174]:
df2['zip_code'] = 0
row_no = 0
# 1331 - 
for i in df1['lat_lng_short']:
    print(row_no)
    df2.loc[df['lat_lng_short'] == i, 'zip_code'] = df1.loc[df1['lat_lng_short'] == i, 'zip_code'][row_no]
#         print(df2.loc[df['lat_lng_short'] == a, 'zip_code'])
    row_no += 1
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


In [176]:
df2.isnull().sum()

no                         0
ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    523467
start_station_id      523781
end_station_name      567268
end_station_id        567501
start_lat                  0
start_lng                  0
end_lat                 4821
end_lng                 4821
member_casual              0
riding_days                0
riding_time                0
start_lat_short            0
start_lng_short            0
lat_lng_short              0
zip_code                   0
dtype: int64

In [184]:
len(df2[df2['riding_days'] > 0])

3967

In [191]:
df2 = df2.loc[df2['riding_days'] == 0]
print(len(df), len(df2))

5136261 5132294


In [192]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5132294 entries, 0 to 5136260
Data columns (total 20 columns):
 #   Column              Dtype  
---  ------              -----  
 0   no                  int64  
 1   ride_id             object 
 2   rideable_type       object 
 3   started_at          object 
 4   ended_at            object 
 5   start_station_name  object 
 6   start_station_id    object 
 7   end_station_name    object 
 8   end_station_id      object 
 9   start_lat           float64
 10  start_lng           float64
 11  end_lat             float64
 12  end_lng             float64
 13  member_casual       object 
 14  riding_days         int64  
 15  riding_time         object 
 16  start_lat_short     object 
 17  start_lng_short     object 
 18  lat_lng_short       object 
 19  zip_code            object 
dtypes: float64(4), int64(2), object(14)
memory usage: 822.3+ MB


In [194]:
df2 = df2.drop(columns = ['no', 'start_lat_short', 'start_lng_short', 'lat_lng_short'])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5132294 entries, 0 to 5136260
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
 13  riding_days         int64  
 14  riding_time         object 
 15  zip_code            object 
dtypes: float64(4), int64(1), object(11)
memory usage: 665.7+ MB


In [195]:
df2.to_csv('Cyclistic_202010_202109_zip_code.csv')

In [5]:
# df2 = pd.read_csv('Cyclistic_202010_202109_zip_code.csv')
df2 = pd.read_csv('Cyclistic_202010_202109_zip_code_Tableau.csv')
df2.head()

,day_of_week,End Station Id,End Station Name,Ended At,Member Casual,Ride Id,Rideable Type,Riding Time,season,Start Station Id,Start Station Name,Started At,Zip Code,classic_bike,docked_bike,electrict_bike,End Lat,End Lng,F1,Riding Days,riding_time,Start Lat,Start Lng,test
0,Friday,NaN,Central St & Girard Ave,26/03/2021 4:28:58 PM,casual,56A8B5FB254C94A8,classic_bike,30/12/1899 12:02:15 AM,SPRING,NaN,Central St & Girard Ave,26/03/2021 4:26:43 PM,60201,100,0,0,42.064313,-87.686152,1069469,0,2.250000,42.064313,-87.686152,12
1,Sunday,NaN,Chicago Ave & Sheridan Rd,21/03/2021 12:34:04 AM,casual,A590F743136B3155,classic_bike,30/12/1899 12:07:54 AM,SPRING,NaN,Central St & Girard Ave,21/03/2021 12:26:10 AM,60201,100,0,0,42.050491,-87.677821,1120787,0,7.900000,42.064313,-87.686152,12
2,Saturday,NaN,Chicago Ave & Sheridan Rd,27/03/2021 11:57:42 PM,casual,B31A34D7687BCCB6,classic_bike,30/12/1899 12:09:31 AM,SPRING,NaN,Central St & Girard Ave,27/03/2021 11:48:11 PM,60201,100,0,0,42.050491,-87.677821,1120786,0,9.516667,42.064313,-87.686152,12
3,Tuesday,NaN,Chicago Ave & Sheridan Rd,30/03/2021 12:49:23 AM,casual,EC02351300FB9ACA,classic_bike,30/12/1899 12:10:59 AM,SPRING,NaN,Central St & Girard Ave,30/03/2021 12:38:24 AM,60201,100,0,0,42.050491,-87.677821,1120758,0,10.983333,42.064313,-87.686152,12
4,Friday,NaN,Chicago Ave & Washington St,19/03/2021 4:08:50 PM,casual,EBB287674E81F7F2,classic_bike,30/12/1899 12:25:59 AM,WINTER,NaN,Central St & Girard Ave,19/03/2021 3:42:51 PM,60201,100,0,0,42.032562,-87.679101,1075926,0,25.983333,42.064313,-87.686152,12


In [20]:
df2.isnull().sum()

day_of_week                 0
End Station Id        2772035
End Station Name       565018
Ended At                    0
Member Casual               0
Ride Id                     0
Rideable Type               0
Riding Time                 0
season                      0
Start Station Id      2745978
Start Station Name     523426
Started At                  0
Zip Code                    0
classic_bike                0
docked_bike                 0
electrict_bike              0
End Lat                  2661
End Lng                  2661
F1                          0
Riding Days                 0
riding_time                 0
Start Lat                   0
Start Lng                   0
test                        0
dtype: int64

Drop null End Lat and Lng

In [22]:
len(df2)

5132294

In [41]:
len(df2.loc[(df2['End Lat'].isnull()) | (df2['End Lng'].isnull())])

2661

In [42]:
5132294 - 2661

5129633

In [40]:
len(df2.dropna(axis = 0, subset = ['End Lat', 'End Lng']))

5129633

In [43]:
df2 = df2.dropna(axis = 0, subset = ['End Lat', 'End Lng'])
df2.isnull().sum()

day_of_week                 0
End Station Id        2769374
End Station Name       562357
Ended At                    0
Member Casual               0
Ride Id                     0
Rideable Type               0
Riding Time                 0
season                      0
Start Station Id      2744856
Start Station Name     523426
Started At                  0
Zip Code                    0
classic_bike                0
docked_bike                 0
electrict_bike              0
End Lat                     0
End Lng                     0
F1                          0
Riding Days                 0
riding_time                 0
Start Lat                   0
Start Lng                   0
test                        0
dtype: int64

In [7]:
df2.columns

Index(['day_of_week', 'End Station Id', 'End Station Name', 'Ended At',
       'Member Casual', 'Ride Id', 'Rideable Type', 'Riding Time', 'season',
       'Start Station Id', 'Start Station Name', 'Started At', 'Zip Code',
       'classic_bike', 'docked_bike', 'electrict_bike', 'End Lat', 'End Lng',
       'F1', 'Riding Days', 'riding_time', 'Start Lat', 'Start Lng', 'test'],
      dtype='object')

# Define df3 

In [130]:
df3 = df2.loc[:,('Start Lat', 'Start Lng', 'Zip Code', 'End Lat', 'End Lng')].astype(str)
df3['start_lat_lng'] = df3.loc[:, 'Start Lat'].str[:6]+', '+df3.loc[:, 'Start Lng'].str[:7]
df3['end_lat_lng'] = df3.loc[:, 'End Lat'].str[:6]+', '+df3.loc[:, 'End Lng'].str[:7]
df3.head()

,Start Lat,Start Lng,Zip Code,End Lat,End Lng,start_lat_lng,end_lat_lng
0,42.064313,-87.686152,60201,42.064313,-87.686152,"42.064, -87.686","42.064, -87.686"
1,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
2,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
3,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
4,42.064313,-87.686152,60201,42.032562,-87.679101,"42.064, -87.686","42.032, -87.679"


In [133]:
df3['end_lat_lng'].nunique()

6801

In [140]:
# Drop duplicates of end_lat_lng
df3 = df3.drop_duplicates(subset = 'end_lat_lng')
df3 = df3.reset_index()
df3.head()

,index,Start Lat,Start Lng,Zip Code,End Lat,End Lng,start_lat_lng,end_lat_lng
0,0,42.064313,-87.686152,60201,42.064313,-87.686152,"42.064, -87.686","42.064, -87.686"
1,1,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
2,4,42.064313,-87.686152,60201,42.032562,-87.679101,"42.064, -87.686","42.032, -87.679"
3,6,42.063598,-87.69873,60201,42.063598,-87.69873,"42.063, -87.698","42.063, -87.698"
4,10,42.063598,-87.69873,60201,41.96167,-87.65464,"42.063, -87.698","41.961, -87.654"


In [131]:
print(len(df3))
df3.isnull().sum()

5129633


Start Lat        0
Start Lng        0
Zip Code         0
End Lat          0
End Lng          0
start_lat_lng    0
end_lat_lng      0
dtype: int64

In [47]:
df3['start_lat_lng'] = df3.loc[:, 'Start Lat'].str[:6]+', '+df3.loc[:, 'Start Lng'].str[:7]
df3['end_lat_lng'] = df3.loc[:, 'End Lat'].str[:6]+', '+df3.loc[:, 'End Lng'].str[:7]

df3.head()

,Start Lat,Start Lng,Zip Code,End Lat,End Lng,start_lat_lng,end_lat_lng
0,42.064313,-87.686152,60201,42.064313,-87.686152,"42.064, -87.686","42.064, -87.686"
1,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
2,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
3,42.064313,-87.686152,60201,42.050491,-87.677821,"42.064, -87.686","42.050, -87.677"
4,42.064313,-87.686152,60201,42.032562,-87.679101,"42.064, -87.686","42.032, -87.679"


In [48]:
print(df3['start_lat_lng'].nunique(), df3['end_lat_lng'].nunique())

2405 6801


I manually searched every lat and len in df_err_dict. 

Because I think it was the best solution I could come up with at this moment...

In [122]:
crr_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
 '12303 S Ashland Ave, Calumet Park, IL 60827, United States',
 'Uptown, Chicago, IL 60613, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, United States',
 '1736 S Michigan Ave, Chicago, IL 60616, United States',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
 'Douglas, Chicago, IL 60616, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'Major Taylor Trail & 115th St, Chicago, IL 60643, United States',
 '10729 S Ave F, Chicago, IL 60617, USA',
 'Parking lot, Chicago, IL 60616, USA']

['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
 '12303 S Ashland Ave, Calumet Park, IL 60827, United States',
 'Uptown, Chicago, IL 60613, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, United States',
 '1736 S Michigan Ave, Chicago, IL 60616, United States',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
 'Douglas, Chicago, IL 60616, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'Major Taylor Trail & 115th St, Chicago, IL 60643, United States',
 '10729 S Ave F, Chicago, IL 60617, USA',
 'Parking lot, Chicago, IL 60616, USA']

In [85]:
# Add crr_add to df_err_dict
df_err_dict['crr_add'] = crr_add
df_err_dict

,index,lat_lng,text,crr_add
0,13,"41.969107, -87.674215","Ravenswood Ave & Lawrence Ave, North Ravenswoo...","Ravenswood Ave & Lawrence Ave, Chicago, IL 606..."
1,29,"41.67, -87.66","West Pullman, Calumet Park, Calumet Township, ...","12303 S Ashland Ave, Calumet Park, IL 60827, U..."
2,50,"41.96, -87.64","West Montrose Harbor Drive, Uptown, Chicago, L...","Uptown, Chicago, IL 60613, United States"
3,78,"41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...","11251-11249 S Ave B, Chicago, IL 60617, United..."
4,102,"41.857881, -87.624555","70, East 18th Street, South Loop, Near South S...","1736 S Michigan Ave, Chicago, IL 60616, United..."
5,132,"41.963320, -87.679309","Damen Ave & Sunnyside Ave, West Sunnyside Aven...","2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA"
6,146,"41.83, -87.6","Oakland, Chicago, Cook County, Illinois, Unite...","Douglas, Chicago, IL 60616, USA"
7,186,"41.851381, -87.618856","McCormick Place, South Doctor Martin Luther Ki...","301 E Cermak Rd, Chicago, IL 60616, USA"
8,193,"41.852516, -87.626532","Cermak-McCormick Place, East Cermak Road, Near...","12 E Cermak Rd, Chicago, IL 60616, USA"
9,209,"41.684958, -87.645342","Major Taylor Trail, Placerdale, West Pullman, ...","Major Taylor Trail & 115th St, Chicago, IL 606..."


(1) re.search('.60', df_err_dict['crr_add'][0]).span() says that its zip code position is (42, 45),
but (2) the actual position is (43, 48), so add 1 to start and 3 to end.

(1)
* re.search('.60', df_err_dict['crr_add'][0]).span()[0] --> output = (42)
* re.search('.60', df_err_dict['crr_add'][0]).span()[1] --> output = (45)

(2) the actual (43:48), start +1, end +3
* re.search('.60', df_err_dict['crr_add'][0]).span()[0]+1 --> output = (42+3) = (45)
* re.search('.60', df_err_dict['crr_add'][0]).span()[1]+3 --> output = (45+3) = (48)




In [123]:
# Check if the start and end position are right
print(df_err_dict['crr_add'][0][42:45])
print(df_err_dict['crr_add'][0][43:48]) # (42, 45), actual (43:48), start +1, end +3

 60
60640


In [116]:
zip_code = []

# Find the zip code position in the address
for i in range(len(df_err_dict['crr_add'])):
    if re.search('.60', df_err_dict['crr_add'][i]) != None:
        # Extract start and end position
        start = re.search('.60', df_err_dict['crr_add'][i]).span()[0]+1 # start
        end = re.search('.60', df_err_dict['crr_add'][i]).span()[1]+3 # end
        # Add to the list
        zip_code.append(df_err_dict['crr_add'][i][start:end])

zip_code

['60640',
 '60827',
 '60613',
 '60617',
 '60616',
 '60625',
 '60616',
 '60616',
 '60616',
 '60643',
 '60617',
 '60616']

In [121]:
df_err_dict['zip_code'] = zip_code
df_err_dict

,index,lat_lng,text,crr_add,zip_code
0,13,"41.969107, -87.674215","Ravenswood Ave & Lawrence Ave, North Ravenswoo...","Ravenswood Ave & Lawrence Ave, Chicago, IL 606...",60640
1,29,"41.67, -87.66","West Pullman, Calumet Park, Calumet Township, ...","12303 S Ashland Ave, Calumet Park, IL 60827, U...",60827
2,50,"41.96, -87.64","West Montrose Harbor Drive, Uptown, Chicago, L...","Uptown, Chicago, IL 60613, United States",60613
3,78,"41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...","11251-11249 S Ave B, Chicago, IL 60617, United...",60617
4,102,"41.857881, -87.624555","70, East 18th Street, South Loop, Near South S...","1736 S Michigan Ave, Chicago, IL 60616, United...",60616
5,132,"41.963320, -87.679309","Damen Ave & Sunnyside Ave, West Sunnyside Aven...","2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA",60625
6,146,"41.83, -87.6","Oakland, Chicago, Cook County, Illinois, Unite...","Douglas, Chicago, IL 60616, USA",60616
7,186,"41.851381, -87.618856","McCormick Place, South Doctor Martin Luther Ki...","301 E Cermak Rd, Chicago, IL 60616, USA",60616
8,193,"41.852516, -87.626532","Cermak-McCormick Place, East Cermak Road, Near...","12 E Cermak Rd, Chicago, IL 60616, USA",60616
9,209,"41.684958, -87.645342","Major Taylor Trail, Placerdale, West Pullman, ...","Major Taylor Trail & 115th St, Chicago, IL 606...",60643


In [259]:
# copy df1
df2 = df1[:]

In [234]:
import pdb

In [ ]:
df2['zip_code'].value_counts().sum()

In [336]:
# Change the wrong data in df2 to the correct data in df_err_dict
add = 0
for i in df_err_dict['index']:
#     pdb.set_trace()
    df2['zip_code'].loc[df2.index == i] = df_err_dict['zip_code'].loc[df_err_dict['index'] == i][add]
    add += 1

In [372]:
df['lat_lng_short'].nunique()

2405

In [379]:
df_set = set(df['lat_lng_short'])
df2_set = set(set(df2['lat_lng_short']))

len(df2_set & df_set) # set intersection

330

In [381]:
# prints the missing and additional elements in list1
# print("Missing values in first list:", (set(df2_set).difference(df_set)))
print("Additional values in first list:", (set(df_set).difference(df2_set)))

Additional values in first list: {'41.780, -87.615', '41.948, -87.740', '41.89, -87.6', '41.906, -87.634', '41.958, -87.649', '41.883, -87.626', '41.884, -87.638', '41.872, -87.746', '41.84, -87.76', '41.956, -87.669', '41.780, -87.591', '41.7, -87.71', '41.895, -87.634', '41.939, -87.645', '41.878, -87.649', '41.64, -87.62', '41.895, -87.627', '41.814, -87.597', '41.7, -87.72', '41.867, -87.622', '41.799, -87.587', '41.97, -87.71', '41.854, -87.663', '41.84, -87.77', '41.896, -87.631', '41.85, -87.61', '41.68, -87.57', '41.69, -87.54', '41.858, -87.656', '41.853, -87.620', '41.94, -87.8', '41.877, -87.634', '41.895, -87.624', '41.879, -87.638', '41.887, -87.620', '41.896, -87.621', '41.82, -87.66', '41.897, -87.622', '41.89, -87.65', '41.881, -87.639', '41.904, -87.631', '41.929, -87.642', '41.81, -87.64', '41.7, -87.54', '41.874, -87.627', '41.94, -87.74', '42.01, -87.7', '41.912, -87.634', '41.880, -87.617', '41.87, -87.69', '41.92, -87.71', '41.893, -87.639', '41.903, -87.683', '41

In [47]:
print(len(df), len(df_lat_lng))

5136261 5136261


In [151]:
geo_loc = Nominatim(user_agent="GetLoc")

In [39]:
loc_list = []

# Get location with geocode
# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")
geo_loc.geocode(Latitude+","+Longitude)
# geo_loc.geocode(df['start_lat'][1].astype(str)[:10]+","+df['start_lng'][1].astype(str)[:11])


Location(CVS Pharmacy, 3637, North Southport Avenue, Wrigleyville, Lake View, Chicago, Lake View Township, Cook County, Illinois, 60613, United States, (41.948237, -87.66368319241383, 0.0))

In [98]:
geo_loc.geocode('41.87, -87.63')

Location(960-996, South Park Terrace, Dearborn Park, Loop, Chicago, Cook County, Illinois, 60605, United States, (41.870180399999995, -87.63028242517899, 0.0))

In [ ]:
loc_list = []
# 1913 - 
for i in range(len(df_lat_lng)):
#     zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address[-20:-15]
    zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address
    loc_list.append(zip_code)

In [99]:
loc_list = []
# 1913 - 
for i in range(len(df_lat_lng)):
#     zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address[-20:-15]
    zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address
    loc_list.append(zip_code)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=41.830115%2C+-87.670379&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

1412

In [ ]:
zip_code = []

# Find the zip code position in the address
for i in range(len(df_err_dict['crr_add'])):
    if re.search('.60', df_err_dict['crr_add'][i]) != None:
        
        # Find the zip code position in the address
#         print(re.search('.60', df_err_dict['crr_add'][i]).span())
        
        # Extract start and end position
        start = re.search('.60', df_err_dict['crr_add'][i]).span()[0]+1 # start
        end = re.search('.60', df_err_dict['crr_add'][i]).span()[1]+3 # end
        
        zip_code.append(df_err_dict['crr_add'][i][start:end])

zip_code

In [ ]:
crr_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
 '12303 S Ashland Ave, Calumet Park, IL 60827, United States',
 'Uptown, Chicago, IL 60613, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, United States',
 '1736 S Michigan Ave, Chicago, IL 60616, United States',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
 'Douglas, Chicago, IL 60616, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'Major Taylor Trail & 115th St, Chicago, IL 60643, United States',
 '10729 S Ave F, Chicago, IL 60617, USA',
 'Parking lot, Chicago, IL 60616, USA']

In [ ]:
df_lat_lng1 = df_lat_lng[:802]
df_lat_lng2 = df_lat_lng[802:1605]
df_lat_lng3 = df_lat_lng[1605:]

print(len(df_lat_lng1),
      len(df_lat_lng2),
      len(df_lat_lng3)
     )

# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")

loc_list1 = []
loc_list2 = []
loc_list3 = []

# As df_lat_lng1 start from 0, I don't need to set the range specifically
for i in range(len(df_lat_lng1)):
    zip_code = geo_loc.geocode(df_lat_lng1['lat_lng_short'][i]).address
    loc_list1.append(zip_code)
print('df_lat_lng1 - Done')

# Set the range as df_lat_lng2 starts from 802 and ends at 1604
for j in range(df_lat_lng2['lat_lng_short'].index.start, df_lat_lng2['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng2['lat_lng_short'][j]).address
    loc_list2.append(zip_code)
print('df_lat_lng2 - Done')

# Set the range as df_lat_lng3 starts from 1605 and ends at 2404
for k in range(df_lat_lng3['lat_lng_short'].index.start, df_lat_lng3['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng3['lat_lng_short'][k]).address
    loc_list3.append(zip_code)
print('df_lat_lng3 - Done')


In [ ]:
wr_dict_new_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
                   '12303 S Ashland Ave, Calumet Park, IL 60827, USA',
 'Uptown, Chicago, IL 60613, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
                   'West Town, Chicago, IL 60651, USA',
 '3155 S Lake Shore Dr, Chicago, IL 60616, United States',
 'Parking lot, Chicago, IL 60605, USA',
 '4705 N Damen Ave, Chicago, IL 60625, USA',
 '4108 N Lincoln Ave, Chicago, IL 60618, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 'Millennium Park, Chicago, IL 60601, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'MLK Jr Dr & 56th St, Chicago, IL 60612, United States',
 'Parking lot, Chicago, IL 60605, USA',
                   'Major Taylor Trail & 115th St, Chicago, IL 60643, USA',
                   'Smith Park, Chicago, IL 60612, USA',
 '2130 S Wentworth Ave, Chicago, IL 60616, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '10733 S Ave F, Chicago, IL 60617, USA',
 'Armour Square, Chicago, IL 60616, USA',
 'E 47th St, Chicago, IL 60653, USA',
                   'Smith Park, Chicago, IL 60612, USA',
 '72 W Monroe St, Chicago, IL 60603, United States',
 '5410 N Damen Ave, Chicago, IL 60625, USA',
 '3050 Moe Dr, Chicago, IL 60616, USA',
 'Baseball Junior/Softball, Chicago, IL 60615, USA',
 'Bronzeville, Chicago, IL 60653, USA',
 'State St & 95th St, Chicago, IL 60619, United States',
 '5074 S Calumet Ave, Chicago, IL 60612, USA',
 'Chicago, IL 60616, United States',
 'East Side, Chicago, IL 60617, USA',
 '100 W Randolph St 2nd fl, Chicago, IL 60601, United States',
                   '2910 W Ferdinand St, Chicago, IL 60612, USA',
 '1736 S Michigan Ave, Chicago, IL 60616, USA',
 '4802 N Ravenswood Ave, Chicago, IL 60612, USA',
                   'Parking lot, Chicago, IL 60605, USA',
 'Chicago, IL 60640, United States',
 'Dan Ryan Woods Trails, Chicago, IL 60620, USA',
 '2150 W Lawrence Ave, Chicago, IL 60625, USA',
 'Damen Ave & Leland Ave, Chicago, IL 60625, United States',
 'Chicago, IL 60605, United States',
 'McCormick Place, Chicago, IL 60616, United States',
 'Wells St & 19th St, Chicago, IL 60616, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 'Lincoln Ave & Sunnyside Ave, Chicago, IL 60625, United States',
 '1719 S Clinton St, Chicago, IL 60616, USA',
 'Museum Campus, Chicago, IL 60605, USA',
 'Parking lot, Chicago, IL 60616, USA',
 'Chicago, IL 60633, United States',
 '338 E 51st St, Chicago, IL 60615, USA',
 'S Wolf Lake Blvd, Chicago, IL 60633, United States',
 'Chicago, IL 60633, United States',
 '5420-5400 N Hoyne Ave, Chicago, IL 60625, USA',
 '1400-1414 Michigan Ave, Chicago, IL 60605, USA',
 'Chicago, IL 60633, USA']